In [1]:
from google.colab import auth
auth.authenticate_user()
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=LYafkrF4CS2Bwh0TtodYrbKGbfzoCsslC8mG6BOEGv8&tc=3BJIBxTIL-2F4qAHFRmNAeXFxChRkHlAMSB0fUKB-PU&cc=iA7zrI6Lj0TBaeT3kB77TWPtcGCQBvTAE_cmmXr0Gx8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhWv7dRw0z6S7a_0YB2QuiGZszaGVzz9a0pghYRIkwbDrmTaKo0Cdqc

Successfully saved authorization token.


In [ ]:
# !pip install segmentation-models
# import segmentation_models as sm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# model_path_tf = '/content/drive/MyDrive/l8_sent_collection2_079_128.tf'
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_0_128.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_0_global_norm.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_0_3_norm_cutoff.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_0_3_direction.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_0_3_norm_cutoff_512.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_crop_dnbr_0_two_long_128_crop_mtn.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_crop_VI_0_two_128.tf"
# model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_dnbr_full_norm.tf"
model_path_tf = "/content/drive/MyDrive/l8_sent_collection2_crop_dnbr_0_two_long_128_crop_mtn.tf"
model_path_tf = "/content/drive/MyDrive/l8_collection2_dnbr_five_128_2d_ds.tf"
model_path_tf = "/content/drive/MyDrive/l8_collection2_dnbr_one_128_2d_ds_proj.tf"
model_path_tf = "/content/drive/MyDrive/modis_dnbr_0_one_plus_128_2d_ds.tf"


# model_path_tf = "/content/drive/MyDrive/modis_dnbr_0_five_plus_128_3d_crop_no_ds.tf"








# model.save(model_path_tf, save_format='tf')

In [6]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(model_path_tf, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "output"}) + "'"
print(input_dict)
print(output_dict)

'{"serving_default_input_1:0": "array"}'
'{"StatefulPartitionedCall:1": "output"}'


In [7]:
# Put the EEified model next to the trained model directory.
# EEIFIED_DIR = '/content/drive/MyDrive/EmptyUnet2_test_eeify.tf'
# EEIFIED_DIR = '/content/drive/MyDrive/l8_sent_collection2_079_128_eeify.tf'
# EEIFIED_DIR = '/content/drive/MyDrive/l8_sent_collection2_0_128_eeify.tf'
# EEIFIED_DIR = '/content/drive/MyDrive/l8_sent_collection2_0_global_norm_eeify.tf'
# EEIFIED_DIR = '/content/drive/MyDrive/l8_sent_collection2_0_3_direction_eeify.tf'
# EEIFIED_DIR = 'content/drive/MyDrive/l8_sent_collection2_0_3_norm_cutoff_512_eeify.tf'
EEIFIED_DIR = 'content/drive/MyDrive/modis_dnbr_0_one_plus_128_2d_ds_eeify.tf'
# shapefile.transform(targetProjection);
PROJECT = 'gee-serdp-upload'

!gcloud config set project {PROJECT}
# You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {model_path_tf} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

Updated property [core/project].
Successfully saved project id
Success: model at 'content/drive/MyDrive/modis_dnbr_0_one_plus_128_2d_ds_eeify.tf' is ready to be hosted in AI Platform.


In [8]:
%%writefile config.yaml
autoScaling:
  minNodes: 10

Writing config.yaml


In [9]:
REGION = 'us-east4'
# MODEL_NAME = 'Empty_Unet2_test'
# MODEL_NAME = 'l8_sent_collection2_079_128_eeify_test'
# MODEL_NAME = 'l8_sent_collection2_0_128_eeify_test2'
# MODEL_NAME = 'l8_sent_collection2_0_global_norm_eeify'
# MODEL_NAME = 'l8_sent_collection2_0_3_direction_eeify'
# MODEL_NAME = 'l8_sent_collection2_0_3_norm_cutoff_512_eeify'
MODEL_NAME = 'modis_dnbr_0_one_plus_128_2d_ds_eeify'

VERSION_NAME = 'v0'
STAGING_BUCKET = 'gs://smp-scratch'

!gcloud ai-platform models create {MODEL_NAME} \
  --project {PROJECT} \
  --region {REGION}

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --region {REGION} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --staging-bucket {STAGING_BUCKET} \
  --framework "TENSORFLOW" \
  --runtime-version=2.3 \
  --python-version=3.7

Using endpoint [https://us-east4-ml.googleapis.com/]
Created ai platform model [projects/gee-serdp-upload/models/modis_dnbr_0_one_plus_128_2d_ds_eeify].
Using endpoint [https://us-east4-ml.googleapis.com/]
